In [3]:
import pandas as pd
import matplotlib
from matplotlib import pylab as plt
import numpy as np
from IPython.display import display


def counted(list_of_numbers):
    hashed = {}
    
    for i in list_of_numbers:
        if i not in hashed:
            hashed[i] = 0
        hashed[i] += 1
        
    return [(k, v) for k, v in hashed.iteritems()]

def initialize():
    MAX_COLUMNS_DISPLAY = 200
    MAX_ROWS_DISPLAY = 10
    pd.set_option("display.max_columns", MAX_COLUMNS_DISPLAY)
    pd.set_option("display.max_rows", MAX_ROWS_DISPLAY)


def phi(X):
    return map(lambda x: x ** 2, X)


def teoretical_function(x):
    return x**0.5 / 6.0

def teoretical_density(x):
    return 1.0 / 12.0 / x**0.5

def add_fake_points(Fx, Fy):
    Fx = [-1000000.0] + Fx + [1000000.0]
    Fy = [0.0] + Fy + [1.0]
    
    return (Fx, Fy)

def add_dencity_fake_points(Fx, Fy):
    Fx = [-1000] + Fx + [1000]
    Fy = [0] + Fy + [0]
    
    return (Fx, Fy)


def add_histogramm_fake_points(start_point_x, Fx, Fy):
    Fx = [start_point_x] + Fx + [Fx[-1]]
    Fy = [0] + Fy + [0]
    
    return (Fx, Fy)
    


def main():
    initialize()
    a, b = 0, 6

    n = int(raw_input("Enter n = "))

    eps = np.random.random(n)    
    X = eps * (b - a) + a
    Y = phi(X)

    print "\nInput data:"
    input_data_frame = pd.DataFrame({"Xi": X, "Yi = phi(Xi)": Y}).transpose()
    display(input_data_frame)
    
    print "\nVariation series:"
    variation_series = sorted(Y)
    variation_series_data_frame = pd.DataFrame({"Variation series": variation_series}).transpose()
    display(variation_series_data_frame)
    
    
    counted_series = sorted(counted(variation_series))
    
# Practice Function of distribution
    Fsize = len(counted_series)
    Fx_list, Fy_list = [], []
    F_sum = 0
    
    for i in counted_series:
        F_sum += float(i[1]) / float(n)
        Fx_list.append(i[0])
        Fy_list.append(F_sum)
    Fx_list, Fy_list = add_fake_points(Fx_list, Fy_list)
    
    plt.axis([-5, 1.5 * Fx_list[-2], -1, 1.5])
    plt.step(list(reversed(Fx_list)), list(reversed(Fy_list))) # To make function continuous

# Teoretical Function of distribution
    
    TFx_list = list(matplotlib.mlab.frange(a**2, b**2, 0.01))
    TFy_list = map(teoretical_function, TFx_list)
    
    TFx_list, TFy_list = add_fake_points(TFx_list, TFy_list)
    plt.plot(list(reversed(TFx_list)), list(reversed(TFy_list)))
    plt.show()
    
# Histogramm Drawing by equal probability method

    if n <= 100:
        M = int(n**0.5)
    else:
        M = int(np.log(n) * 2)
        
    #while n % M != 0:
    #    M += 1
        
    delta_volume = n / M
    A = [variation_series[0]]
    B = []
    
    for i in xrange(M):
        if (i != 0):
            A.append(B[-1])
        if (i != M - 1):
            B.append((variation_series[(i + 1) * delta_volume - 1] \
                  + variation_series[(i + 1) * delta_volume]) / 2.0)
    B.append(variation_series[-1])
            
    Mi = [0] * M
    current_interval = 0
    for x in variation_series:
        if x > B[current_interval] and current_interval + 1 < M:
            current_interval += 1
            
        Mi[current_interval] += 1

    histogram_x = []
    histogram_y = []
    
    
    for i in xrange(M):
        histogram_x.append(B[i])
        histogram_y.append(float(Mi[i]) / float(len(variation_series)) / (B[i] - A[i]))
        
    histogram_x, histogram_y = add_histogramm_fake_points(A[0], histogram_x, histogram_y)

    plt.step(histogram_x, histogram_y)
    
# Distribution Polygon Drawing
    dpolygon_x = []
    dpolyhon_y = []
    
    for i in xrange(M):
        dpolygon_x.append((A[i] + B[i]) / 2.0)
        dpolyhon_y.append(float(Mi[i]) / float(len(variation_series)) / (B[i] - A[i]))
        
    plt.plot(dpolygon_x, dpolyhon_y)
    
# Distribution Function on a grouped data

    GFx_list = []
    GFy_list = []
    GF_sum, current = 0, 0
    
    for i in xrange(M):
        while current < n and (variation_series[current] <= B[i] \
                or i == M - 1):
            GF_sum += 1
            current += 1
        
        GFx_list.append(B[i])
        GFy_list.append(float(GF_sum) / float(n))
        
    GFx_list, GFy_list = add_fake_points(GFx_list, GFy_list)
    #plt.axis([-5, 1.5 * GFx_list[-2], -1, 1.5])
    #plt.step(list(reversed(GFx_list)), list(reversed(GFy_list)))
    
# Teoretical Density Function
    TDx_list = list(matplotlib.mlab.frange(a**2, b**2, 0.01))
    TDy_list = map(teoretical_density, TDx_list)
    
    TDx_list, TDy_list = add_dencity_fake_points(TDx_list, TDy_list)
    plt.plot(TDx_list, TDy_list)
    
    plt.axis([-5, 1.5 * TDx_list[-2], 0.0, 0.5])
    plt.show()
    
# Calculate Pearston hi-squared test
    P_probability = []
    T_probability = []
    square_hi = 0.0

    for i in xrange(M):
        P_probability.append(float(Mi[i]) / float(n))
        T_probability.append(teoretical_function(B[i]) - teoretical_function(A[i]))
        #print "practice p = {}, teoretical p = {}".format(P_probability[-1], T_probability[-1])
        square_hi += float(n) * (P_probability[-1] - T_probability[-1])**2 / T_probability[-1]
        #print square_hi
        
    print "Squared hi = {} where k = {}".format(square_hi, M - 1)   
    
    
# Calculate Kolmagorov test
    max_diff = 0.0
    
    ind = 1
    for k, v in counted_series:
        max_diff = max(max_diff, abs(teoretical_function(k) - \
                                    Fy_list[ind]))
        ind += 1
        
    Lambda = float(n)**0.5 * max_diff
    print "Kolmagorov lambda = {}".format(Lambda)
    
# Calculate Richard von Mises test

    misis_test = 0.0
    
    for i in xrange(len(variation_series)):
        misis_test += (teoretical_function(variation_series[i]) - float(i + 0.5) / float(n))**2
    misis_test += 1 / float(12 * n)
    
    print "Misis statistic = {}".format(misis_test)

    
if __name__ == "__main__":
    main()


Enter n = 50

Input data:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
Xi,3.550302,0.567119,1.104286,3.742744,2.975587,1.222025,4.296688,0.559837,0.575895,1.077465,1.743205,4.164144,3.304790,2.894025,0.652995,5.956191,4.734139,4.961797,0.404276,3.358324,1.330720,4.731462,3.161956,5.910937,1.001506,5.61332,1.464926,5.597212,2.000028,0.048426,1.473767,3.889905,0.266272,1.848600,2.524844,3.195263,3.535340,0.856453,2.654633,3.427121,4.227975,1.210490,5.998041,5.790802,3.715566,1.827943,4.467278,5.766984,2.829223,5.933089
Yi = phi(Xi),12.604643,0.321624,1.219449,14.008136,8.854118,1.493345,18.461527,0.313417,0.331655,1.160930,3.038764,17.340091,10.921636,8.375381,0.426402,35.476209,22.412077,24.619427,0.163439,11.278339,1.770815,22.386733,9.997964,34.939173,1.003013,31.50936,2.146008,31.328782,4.000112,0.002345,2.171988,15.131362,0.070901,3.417321,6.374835,10.209706,12.498629,0.733511,7.047077,11.745158,17.875774,1.465285,35.976499,33.533391,13.805432,3.341375,19.956570,33.258100,8.004505,35.201544



Variation series:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
Variation series,0.002345,0.070901,0.163439,0.313417,0.321624,0.331655,0.426402,0.733511,1.003013,1.16093,1.219449,1.465285,1.493345,1.770815,2.146008,2.171988,3.038764,3.341375,3.417321,4.000112,6.374835,7.047077,8.004505,8.375381,8.854118,9.997964,10.209706,10.921636,11.278339,11.745158,12.498629,12.604643,13.805432,14.008136,15.131362,17.340091,17.875774,18.461527,19.95657,22.386733,22.412077,24.619427,31.328782,31.50936,33.2581,33.533391,34.939173,35.201544,35.476209,35.976499


Squared hi = 3.84200444217 where k = 6
Kolmagorov lambda = 0.525891055218
Misis statistic = 0.0623684086523
